<a href="https://colab.research.google.com/github/Jayraj25/titanicDataSurvivalPrediction/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install pandas 

In [0]:
from google.colab import drive
drive.mount('/Titanic')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /Titanic


In [0]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [0]:
! ls "/Titanic/My Drive"

'27 culinery '
 aadhar1.pdf
 aadhar.pdf
 Capture.PNG
 Classroom
 client.c
 cll.c
'Colab Notebooks'
'Dead Reckoning with X-Y-Z parameters.gslides'
 dll.c
'IELTS & GRE'
'IELTS & GRE.zip'
'JayrajThakor_InternshalaResume (2).pdf'
 letusc-yashwantkanetkar.pdf
 ms.c
 MYCV.pdf
 Paperpresentation.pptx
 Project.zip
 sll.c
 TechnoVIT.mp4
 TitanicData
 Titanic.ipynb


In [0]:
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
import xgboost
from sklearn.model_selection import train_test_split
df = pd.read_csv(io.BytesIO(uploaded['train.csv']))
df_test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [0]:
def preprocessing(df):
  df.fillna({"Age":df["Age"].mean(),"Embarked":df["Embarked"].mode()[0]},inplace=True)
  df1 = df.drop(["Cabin"],axis=1)
  le = LabelEncoder()
  var_mod = ['Sex','Pclass','Embarked']
  for i in var_mod:
    df1[i] = le.fit_transform(df1[i])

  df1 = pd.get_dummies(df1,columns=['Sex','Pclass','Embarked'])
  df_survived = df1['Survived']
  df1 = df1.drop(['Survived','Name','Ticket'],axis=1)
  df = pd.concat([df1,df_survived],axis=1)
  return df

def preprocess_test(df):
  df.fillna({"Age":df["Age"].mean(),"Embarked":df["Embarked"].mode()[0],"Fare":df["Fare"].mean()},inplace=True)
  df1 = df.drop(["Cabin"],axis=1)
  le = LabelEncoder()
  var_mod = ['Sex','Pclass','Embarked']
  for i in var_mod:
    df1[i] = le.fit_transform(df1[i])

  df1 = pd.get_dummies(df1,columns=['Sex','Pclass','Embarked'])
  df1 = df1.drop(['Name','Ticket'],axis=1)
  return df1

In [0]:
df_train = preprocessing(df)
df_test = preprocess_test(df_test)

In [0]:
df_train
#df_test

,PassengerId,Age,SibSp,Parch,Fare,Sex_0,Sex_1,Pclass_0,Pclass_1,Pclass_2,Embarked_0,Embarked_1,Embarked_2,Survived
0,1,22.000000,1,0,7.2500,0,1,0,0,1,0,0,1,0
1,2,38.000000,1,0,71.2833,1,0,1,0,0,1,0,0,1
2,3,26.000000,0,0,7.9250,1,0,0,0,1,0,0,1,1
3,4,35.000000,1,0,53.1000,1,0,1,0,0,0,0,1,1
4,5,35.000000,0,0,8.0500,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,27.000000,0,0,13.0000,0,1,0,1,0,0,0,1,0
887,888,19.000000,0,0,30.0000,1,0,1,0,0,0,0,1,1
888,889,29.699118,1,2,23.4500,1,0,0,0,1,0,0,1,0
889,890,26.000000,0,0,30.0000,0,1,1,0,0,1,0,0,1


In [0]:
df = df_train.iloc[:,:-1]
y = df_train.iloc[:,13:14]
y

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [0]:
X_train,X_test,y_train,y_test = train_test_split(df,y,test_size=0.3,random_state=0)
classifiers = []
model1 = xgboost.XGBClassifier()
classifiers.append(model1)
model2 = svm.SVC()
classifiers.append(model2)
model3 = tree.DecisionTreeClassifier()
classifiers.append(model3)
model4 = RandomForestClassifier()
classifiers.append(model4)
model5 = LogisticRegression()
classifiers.append(model5)

In [0]:
#df_test5
#X_test
df_test.isna().sum()
df_test.shape

(418, 13)

In [0]:
for model in classifiers:
  model.fit(X_train,y_train)
  y_pred_model = model.predict(X_test)
  #y_pred = model.predict(df_test)
  #submission = pd.DataFrame({'PassengerId':df_test['PassengerId'],'Survived':y_pred})
  acc = accuracy_score(y_test, y_pred_model)
  print("Accuracy of %s is %s"%(model, acc))
  cm = confusion_matrix(y_test, y_pred_model)
  print("Confusion Matrix of %s is %s"%(model, cm))
  #print("Accuracy of logistic regression classifier on test set: {:.4f}".format(model.score(X_test, y_pred[0:268])))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using

Accuracy of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1) is 0.832089552238806
Confusion Matrix of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1) is [[15

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
model1.fit(X_train,y_train)
#y_pred_model = model.predict(X_test)
y_pred = model1.predict(df_test)
submission = pd.DataFrame({'PassengerId':df_test['PassengerId'],'Survived':y_pred})
filename = 'Titanic Predictions 2.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Titanic Predictions 2.csv


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
files.download(filename)